Format Twitter data to match the format of Google Trends data

In [859]:
# !pip install scikit-learn

In [860]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [861]:
def get_weeks(year_start,year_end):
    # The date range that we query
    datelist = pd.date_range(f'{year_start}-01-01', f'{year_end+1}-01-01').to_series()

    # Extract year and week 
    year = datelist.apply(lambda x: x.strftime('%Y')).astype(int)
    week = datelist.apply(lambda x: x.strftime('%U')).astype(int)

    # Combine years and their weeks in one table
    data_years = pd.DataFrame(data=[year, week], index=['year', 'week']).T

    # Get number of weeks per year
    data_weeks = data_years.groupby(['year']).max().to_dict('index')

    # Get a list of weeks per year
    week_ranges=[]
    for year in data_weeks.keys():
        week_ranges.append(list(range(1,data_weeks[year]['week']+1)))

    # Create a dict with year and exact week numbers
    weeks_per_year = dict(zip(list(data_weeks.keys()), week_ranges))
    
    return weeks_per_year

In [862]:
def find_last_week_of_year(year):
    weeks_in_year = get_weeks(year,year)
    return max(weeks_in_year[year])

In [863]:
def find_missing_weeks(data):
    year_start = data.year.min()
    year_end = data.year.max()
    
    weeks_per_year=get_weeks(year_start,year_end)
    actual_data = data.groupby('year')['week_number'].apply(set).apply(list)

    weeks_to_add={}
    for year in range(year_start,year_end+1):
        weeks1 = get_weeks(year_start,year_end)[year]
        weeks2 = actual_data[year]
        weeks_to_add[year] = (list(set(weeks1) - set(weeks2)))
        
    return weeks_to_add

In [864]:
def handle_missing_weeks(data):
    weeks_to_add = find_missing_weeks(data)
    
    for year in weeks_to_add.keys():
        for week in weeks_to_add[year]:
            data = data.append({'year':year, 'week_number':week, 'content':0}, ignore_index=True)
            
    return data.sort_values(['year', 'week_number']).reset_index(drop=True)

In [882]:
def get_relative_scores_per_year(data):
    new_data = pd.DataFrame()
    for year in data.year.unique():
        data_year = data[data['year']==year]
        
        scaler = MinMaxScaler()
        data_year['relative_yearly'] = scaler.fit_transform(np.array(data_year.content).reshape(-1, 1))*100
        new_data=pd.concat([new_data, data_year])
        
    return new_data

In [883]:
def handle_week_zero(data):
    data['prev_year']= data.year-1
    data['max_week_prev_year'] = data.prev_year.apply(find_last_week_of_year)
    
    year_update = data.loc[data['week_number'] == 0].prev_year
    week_update = data.loc[data['week_number'] == 0].max_week_prev_year

    
    data.loc[data['week_number'] == 0, 'year']  = year_update.values
    data.loc[data['week_number'] == 0, 'week_number']  = week_update.values
    
    data = data[data.year>=2014] # Match filter of Google Trends query
    
    data = data.groupby(['year','week_number']).sum().reset_index() # Merge rows with week MAX together under the same sum
    return data[['year', 'week_number', 'content']]


In [884]:
def scale_year_range(data, col, year_start, year_end):
    data_period = data[(data.year>=year_start) & (data.year<=year_end)]
    scaler = MinMaxScaler()
    data_period['relative_period'] = scaler.fit_transform(np.array(data_period[col]).reshape(-1, 1))*100
    return data_period

In [885]:
def load_data(drug):
    data=pd.read_csv(f'absolute_tweet_values_{drug}.csv')
    data=data[data.Year>=year_start]
    data = data.rename(columns={'Year': 'year', 'week': 'week_number', 'Tweet': 'content'})
    data=data.reset_index(drop=True)
    return data

In [886]:
year_start=2014
year_end=2023

In [887]:
cocaine=load_data('cocaine')
xtc=load_data('xtc')
ghb=load_data('ghb')

---

#### Handle week 0

In [888]:
cocaine = handle_week_zero(cocaine)
xtc = handle_week_zero(xtc)
ghb = handle_week_zero(ghb)

#### Handle missing weeks 
Make sure to have this in every notebook

In [889]:
cocaine = handle_missing_weeks(cocaine)
xtc = handle_missing_weeks(xtc)
ghb = handle_missing_weeks(ghb)

#### Compute relative scores for drug articles per week per drug

In [890]:
cocaine = get_relative_scores_per_year(cocaine)
xtc = get_relative_scores_per_year(xtc)
ghb = get_relative_scores_per_year(ghb)

#### Scaling on all years 

In [891]:
cocaine = scale_year_range(cocaine, 'content', 2014, 2023)
xtc = scale_year_range(xtc, 'content', 2014, 2023)
ghb = scale_year_range(ghb, 'content', 2014, 2023)

## Save subsets as files

In [892]:
cocaine.to_csv("datasets/cocaine_twitter.csv", index=False)
xtc.to_csv("datasets/xtc_twitter.csv", index=False)
ghb.to_csv("datasets/ghb_twitter.csv")